In [ ]:
import numpy as np
import os
import pickle
import cloudpickle
from astropy.io import fits
from llamas_pyjamas.config import CALIB_DIR, OUTPUT_DIR, LUT_DIR
from llamas_pyjamas.constants import idx_lookup
from llamas_pyjamas.Flat.flatProcessing import produce_flat_extractions, reduce_flat
from llamas_pyjamas.Utils.utils import concat_extractions
from llamas_pyjamas.Arc.arcLlamas import arcTransfer
import llamas_pyjamas.Extract.extractLlamas as extract
from llamas_pyjamas.constants import RED_IDXS, GREEN_IDXS, BLUE_IDXS
import llamas_pyjamas.Arc.arcLlamasMulti as arc
import llamas_pyjamas.GUI.guiExtract as ge

# Generating new wavelength solutions

### 1. selecting the arc file to extract 

In [ ]:
OUTPUT_DIR = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/testing/wavelength_soln_comissioning/'

arc_filename   = '/Users/slh/Library/CloudStorage/Box-Box/slhughes/Llamas_Commissioning_Data/ut20241202-03/LLAMAS_2024-12-03T17_11_57.326_mef.fits'
ge.GUI_extract(arc_filename, output_dir=OUTPUT_DIR)

arc_picklename = os.path.join(OUTPUT_DIR, os.path.basename(arc_filename).replace('_mef.fits', '_extract.pkl'))
with open(arc_picklename, 'rb') as fp:
    batch_data = pickle.load(fp)
arcdict = extract.ExtractLlamas.loadExtraction(arc_picklename)
arcspec, metadata = arcdict['extractions'], arcdict['metadata']

In [ ]:
arc.shiftArcX(arc_picklename)
shift_picklename = arc_picklename.replace('_extract.pkl', '_extract_shifted.pkl')


### Extract the appropriate flat field files

In [ ]:
red_flat_file = '/Users/slh/Library/CloudStorage/Box-Box/slhughes/Llamas_Commissioning_Data/ut20241202-03/LLAMAS_2024-12-02T23_53_52.524_mef.fits'
green_flat_file = '/Users/slh/Library/CloudStorage/Box-Box/slhughes/Llamas_Commissioning_Data/ut20241202-03/LLAMAS_2024-12-02T23_48_34.754_mef.fits'
blue_flat_file = '/Users/slh/Library/CloudStorage/Box-Box/slhughes/Llamas_Commissioning_Data/ut20241202-03/LLAMAS_2024-12-02T23_48_34.754_mef.fits'

In [ ]:
reduce_flat(red_flat_file, RED_IDXS, tracedir=CALIB_DIR, channel='red', save_dir=OUTPUT_DIR)
reduce_flat(green_flat_file, GREEN_IDXS, tracedir=CALIB_DIR, channel='green', save_dir=OUTPUT_DIR)
reduce_flat(blue_flat_file, BLUE_IDXS, tracedir=CALIB_DIR, channel='blue', save_dir=OUTPUT_DIR)

In [ ]:
import cloudpickle
combined_flat_file = os.path.join(OUTPUT_DIR, 'combined_flat_extractions.pkl')
_ex = [os.path.join(OUTPUT_DIR, 'red_extractions_flat.pkl'), os.path.join(OUTPUT_DIR, 'green_extractions_flat.pkl'), os.path.join(OUTPUT_DIR, 'blue_extractions_flat.pkl')]
concat_extractions(_ex, combined_flat_file)

red_extract = extract.ExtractLlamas.loadExtraction(os.path.join(OUTPUT_DIR, 'red_extractions_flat.pkl'))
green_extract = extract.ExtractLlamas.loadExtraction(os.path.join(OUTPUT_DIR, 'green_extractions_flat.pkl'))
blue_extract = extract.ExtractLlamas.loadExtraction(os.path.join(OUTPUT_DIR, 'blue_extractions_flat.pkl'))

print(combined_flat_file)
print(len(red_extract['extractions']))
print(len(green_extract['extractions']))
print(len(blue_extract['extractions']))

import llamas_pyjamas.Extract.extractLlamas as extract # type: ignore

flatdict = extract.ExtractLlamas.loadExtraction(combined_flat_file)
print(len(flatdict['extractions']))

# This sets a reference point to use for relative throughput calculations
flatdict['extractions'][10].relative_throughput[5] = 0.96
flatdict['extractions'][10].relative_throughput

In [ ]:
import cloudpickle
from llamas_pyjamas.constants import idx_lookup
import llamas_pyjamas.Extract.extractLlamas as extract

def reorder_extraction_to_standard(input_pickle, output_pickle=None):
    """
    Reorder an extraction pickle to match the standard LLAMAS extension ordering.
    
    Standard order is: red1A, green1A, blue1A, red1B, green1B, blue1B, ...
    
    Args:
        input_pickle: Path to the input extraction pickle
        output_pickle: Path for output (default: replaces input with '_reordered.pkl')
    
    Returns:
        Path to the reordered pickle file
    """
    # Load the extraction
    data = extract.ExtractLlamas.loadExtraction(input_pickle)
    extractions = data['extractions']
    metadata = data['metadata']
    
    # Build a mapping from (channel, bench, side) -> current index
    current_mapping = {}
    for i, meta in enumerate(metadata):
        key = (meta['channel'].lower(), str(meta['bench']), meta['side'].upper())
        current_mapping[key] = i
    
    # Print current ordering for debugging
    print("Current extension order:")
    for i, meta in enumerate(metadata):
        print(f"  {i}: {meta['channel']} {meta['bench']}{meta['side']} (nfibers={meta['nfibers']})")
    
    # Create new lists in standard order
    new_extractions = []
    new_metadata = []
    
    # Sort by idx_lookup value to get standard order
    sorted_keys = sorted(idx_lookup.keys(), key=lambda k: idx_lookup[k])
    
    print("\nReordering to standard order:")
    for new_idx, key in enumerate(sorted_keys):
        channel, bench, side = key
        lookup_key = (channel.lower(), str(bench), side.upper())
        
        if lookup_key in current_mapping:
            old_idx = current_mapping[lookup_key]
            new_extractions.append(extractions[old_idx])
            new_metadata.append(metadata[old_idx])
            print(f"  {new_idx}: {channel} {bench}{side} <- was index {old_idx}")
        else:
            print(f"  WARNING: {channel} {bench}{side} not found in input!")
    
    # Save the reordered extraction
    if output_pickle is None:
        output_pickle = input_pickle.replace('.pkl', '_reordered.pkl')
    
    # Create new dict structure (include primary_header if present)
    reordered_data = {
        'extractions': new_extractions,
        'metadata': new_metadata
    }
    if 'primary_header' in data:
        reordered_data['primary_header'] = data['primary_header']
    
    # Use cloudpickle like the rest of the codebase
    with open(output_pickle, 'wb') as f:
        cloudpickle.dump(reordered_data, f)
    
    print(f"\nSaved reordered extraction to: {output_pickle}")
    return output_pickle


In [ ]:
# arc.fiberRelativeThroughput(combined_flat_file, shift_picklename)

reordered_flat = reorder_extraction_to_standard(combined_flat_file)
# Then use the reordered file:
arc.fiberRelativeThroughput(reordered_flat, shift_picklename)


In [ ]:
import pickle

with open(combined_flat_file, 'rb') as f:
    flat_data = pickle.load(f)
    
with open(shift_picklename, 'rb') as f:
    arc_data = pickle.load(f)

# Find the 1B extension and compare
for ext, meta in enumerate(flat_data['metadata']):
    if meta.get('bench') == 1 and meta.get('side') == 'B':
        print(f"Extension {ext}: {meta.get('bench')}{meta.get('side')} {meta.get('channel')}")
        print(f"  Flat nfibers: {meta['nfibers']}")
        print(f"  Arc xshift shape: {arc_data['extractions'][ext].xshift.shape[0]}")



In [ ]:
tp=shift_picklename.replace('.pkl','_shifted_tp.pkl')
arc.arcSolve(tp, savefile='test_wavelength_solution.pkl', savedir=OUTPUT_DIR)

### Checking the wavelength solution

In [ ]:
from llamas_pyjamas.Utils.utils import check_reference_arc_wavelength_ranges
ref = os.path.join(OUTPUT_DIR, 'test_wavelength_solution.pkl')
check_reference_arc_wavelength_ranges(ref)